In [1]:
# These are standard modules
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

# This is a custom interface to the pybullet simulator
import ae483_drone

In [2]:
simulator = ae483_drone.Simulator(
    display=True,
    width=640,
    height=480,
)

In [3]:
class RobotClient:
    def __init__(self):
        pass
    
    def run(self, t):
        # Get start time (hard-coded at zero)
        start_time = 0.
        
        # Get current time
        current_time = t
        
        # Define initial point (in the coordinates of the world frame)
        p1 = np.array([0., 0., 0.3])
        
        # Define final point (in the coordinates of the world frame)
        p2 = np.array([0.1, 0., 0.3]) # <-- FIXME (A)
        
        # Define speed in meters / second
        speed = 0.25
        
        # Compute distance from p1 to p2
        distance_from_p1_to_p2 = np.linalg.norm(p1-p2) # <-- FIXME (B)
        
        # Compute time it takes to move from p1 to p2 at desired speed
        time_from_p1_to_p2 = distance_from_p1_to_p2/speed # <-- FIXME (C)
        
        # Compute what fraction of the distance from p1 to p2 should have
        # been travelled by the current time
        s = (current_time - start_time)/time_from_p1_to_p2 # <-- FIXME (D)
        
        # Compute where the drone should be at the current time, in the
        # coordinates of the world frame
        p = ((1-s) * p1) + (s*p2) # <-- FIXME (E)
        
        # Check if the drone should have reached p2 already
        if s >= 1:
            p = p2
        
        # Get x, y, z coordinates of desired position
        o_x = p[0]
        o_y = p[1]
        o_z = p[2]
        psi = 0.
    
        return {
            'o_x': o_x,
            'o_y': o_y,
            'o_z': o_z,
            'psi': psi,
        }

In [4]:
class RobotController:
    def __init__(self):
        pass

    def limitUint16(self, m):
        """
        This function returns the closest integer to "m" in the
        range [0, 65535]. It is called "limitUint16" because an
        "unsigned 16-bit integer" is limited to this range.
        """
        m = np.round(m, decimals=0)
        if m < 0:
            m = 0
        elif m > 65535:
            m = 65535
        return m

    def run(self, state, setpoint):
        # Parse state
        i = 0
        o_x = state['o_x']
        o_y = state['o_y']
        o_z = state['o_z']
        psi = state['psi']
        theta = state['theta']
        phi = state['phi']
        v_x = state['v_x']
        v_y = state['v_y']
        v_z = state['v_z']
        w_x = state['w_x']
        w_y = state['w_y']
        w_z = state['w_z']
        if i == 0:
            alpha_old = 0
            i += 1
        else:
            alpha_old = alpha
        alpha = state['alpha']
        
        # Parse setpoint
        o_x_des = setpoint['o_x']
        o_y_des = setpoint['o_y']
        o_z_des = setpoint['o_z']
        

        alpha_dot = (alpha - alpha_old)/0.01
        print(alpha_dot)
        
        tau_x = 0.00264575 * (o_y - o_y_des) -0.00666529 * phi + 0.00209649 * v_y -0.00109968 * w_x
        tau_y = 0.00223607 * (o_x - o_x_des) -0.01535116 * theta + 0.02127591 * v_x -0.00122116 * w_y + 0.00301966 * alpha + 0.00468941 * alpha_dot
        tau_z = -0.00100000 * psi -0.00102029 * w_z
        f_z = -0.21447611 * (o_z - o_z_des) -0.18939702 * v_z + 0.36297000
        
        # tau_x = 0
        # tau_y = 0
        # tau_z = 0
        # f_z = 0
        print('o_x = ', o_x)
        print('o_x_des = ', o_x_des)
        print('o_y = ', o_y)
        print('o_y_des = ', o_y_des)
        print('phi = ', phi)
        print('v_x = ', v_x)
        print('w_x = ', w_x)
        print('v_y = ', v_y)
        print('alpha = ', alpha)
        print('f_z = ', f_z)

        # FIXME: Replace code here to compute motor power commands
        m_1 = self.limitUint16( -3770739.1 * tau_x -3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_2 = self.limitUint16( -3770739.1 * tau_x + 3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        m_3 = self.limitUint16( 3770739.1 * tau_x + 3770739.1 * tau_y -39494470.8 * tau_z + 122549.0 * f_z )
        m_4 = self.limitUint16( 3770739.1 * tau_x -3770739.1 * tau_y + 39494470.8 * tau_z + 122549.0 * f_z )
        
        return m_1, m_2, m_3, m_4

In [5]:
simulator.clear_drones()

In [6]:
simulator.add_drone(
    'my_drone',
    RobotClient,
    RobotController,
    rgba=[1., 0., 1., 1.],
    m=0.039,   # <-- FIXME
    J_x=1e-5,  # <-- FIXME
    J_y=1e-5,  # <-- FIXME
    J_z=2e-5,  # <-- FIXME
    g=9.81,
    l=0.035,   # <-- FIXME
    k_F=2e-6,  # <-- FIXME
    k_M=1e-8,  # <-- FIXME
)

Set the initial state of this drone. Here, as an example, we start the drone near the desired position that was specified by the client.

In [7]:
simulator.set_state(
    'my_drone',
    {
        'o_x': 0.,
        'o_y': 0.,
        'o_z': 0.3,
        'psi': 0.,
        'theta': 0.,
        'phi': 0.,
        'v_x': 0.,
        'v_y': 0.,
        'v_z': 0.,
        'w_x': 0.,
        'w_y': 0.,
        'w_z': 0.,
    },
)

In [8]:
simulator.get_state(simulator.get_drone_by_name('my_drone')) # debugging

{'o_x': 0.0,
 'o_y': 0.0,
 'o_z': 0.3,
 'psi': 0.0,
 'theta': -0.0,
 'phi': 0.0,
 'v_x': 0.0,
 'v_y': 0.0,
 'v_z': 0.0,
 'w_x': 0.0,
 'w_y': 0.0,
 'w_z': 0.0,
 'alpha': 0.0}

In [9]:
simulator.set_camera_target('my_drone')

In [10]:
simulator.run(
    max_time=20.,
    data_filename='simulation_smooth-2.5m.json',
    video_filename=None,
)

0.0
o_x =  0.0
o_x_des =  0.0
o_y =  0.0
o_y_des =  0.0
phi =  0.0
v_x =  0.0
w_x =  0.0
v_y =  0.0
alpha =  0.0
f_z =  0.36297
0.0
o_x =  0.0
o_x_des =  0.0025
o_y =  0.0
o_y_des =  0.0
phi =  0.0
v_x =  0.0
w_x =  0.0
v_y =  0.0
alpha =  0.0
f_z =  0.3669258956605588
-9.717894184666907e-05
o_x =  2.5034140453976443e-08
o_x_des =  0.005
o_y =  0.0
o_y_des =  0.0
phi =  0.0
v_x =  4.002140659727694e-06
w_x =  0.0
v_y =  0.0
alpha =  -9.717894184666907e-07
f_z =  0.37076572245212613
-0.0004554966654598878
o_x =  1.1705177398216232e-07
o_x_des =  0.0075
o_y =  0.0
o_y_des =  0.0
phi =  0.0
v_x =  1.2307818888770559e-05
w_x =  0.0
v_y =  0.0
alpha =  -4.554966654598878e-06
f_z =  0.37449198736478345
-0.0012592667223567746
o_x =  3.2197477879113246e-07
o_x_des =  0.01
o_y =  0.0
o_y_des =  0.0
phi =  0.0
v_x =  2.5370764841563922e-05
w_x =  0.0
v_y =  0.0
alpha =  -1.2592667223567747e-05
f_z =  0.3781077434344361
-0.0027356195700144782
o_x =  6.940695007986777e-07
o_x_des =  0.0125
o_y =  

error: Not connected to physics server.